In [3]:
import os
import geopandas as gpd
import pandas as pd

In [4]:
import sys
sys.path.append('../')
from process.append import get_enriched_features

In [3]:
california_boundary = gpd.read_file(r'D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb', 
                                    driver='OpenFileGDB', 
                                    layer='California')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [4]:
enriched_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp"

In [5]:
enriched_list = os.listdir(enriched_path)

In [6]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\appended.gdb"

In [7]:
report_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\reports.gdb'

In [8]:
# skip the append, reports gdb file path
skip_list = ['appended.gdb', 'reports.gdb']

In [9]:
point_layers = []
line_layers = []
poly_layers = []
for f in enriched_list:
    if f in skip_list:
        continue
    f_path = enriched_path + r"\{}".format(f)
    gdb_layers = gpd.list_layers(f_path)
    for i in range(len(gdb_layers)):
        if 'point' in gdb_layers.loc[i, 'geometry_type'].lower():
            point_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        elif 'line' in gdb_layers.loc[i, 'geometry_type'].lower():
            line_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
        else:
            poly_layers.append({'gdb_path': f_path, 'layer_name':gdb_layers.loc[i, 'name']})
            
enriched_layers = {'point': point_layers, 
                  'line': line_layers,
                  'polygon': poly_layers}

In [10]:
enriched_layers['line']

[{'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\tmp\\CalTRANS_1950_2025.gdb',
  'layer_name': 'CalTRANS_enriched_20250512'},
 {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\tmp\\CNRA_2020_2025.gdb',
  'layer_name': 'CNRA_enriched_20250421_line'}]

In [11]:
# one time change for activity category

In [12]:
from utils.category import categorize_activity
from utils.standardize_domains import standardize_domains
from utils.counts_to_mas import counts_to_mas

In [13]:
pfirs_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\PFIRS_2020_2025.gdb"
layer_name = 'PFIRS_20250402'

In [13]:
for lyr_type in enriched_layers.keys():
    for path_dict in enriched_layers[lyr_type]:
        gdf = gpd.read_file(path_dict['gdb_path'], driver='OpenFileGDB', layer=path_dict['layer_name'])
        gdf = counts_to_mas(standardize_domains(categorize_activity(gdf)), 1950, 2025)

        #if ('BLM' in path_dict['layer_name']) or ('NFPORS' in path_dict['layer_name']) or ('NPS' in path_dict['layer_name']):
        #    print(path_dict['layer_name'])
        #    gdf.loc[gdf['ACTIVITY_START'] >= f'2023-10-01', 'COUNTS_TO_MAS'] = 'NO'
        print(path_dict['layer_name'])
        gdf.to_file(path_dict['gdb_path'], driver='OpenFileGDB', layer=path_dict['layer_name'])
    

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-06-09 12:17:47,407 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-06-09 12:17:47,408 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-06-09 12:17:47,408 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-06-09 12:17:47,410 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-06-09 12:17:47,411 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-06-09 12:17:47,411 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-06-09 12:17:47,412 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category i

NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD
TBD


2025-06-09 12:17:47,565 INFO  [pyogrio._io           ]  Created 1,985 records
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-06-09 12:17:47,628 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-06-09 12:17:47,629 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-06-09 12:17:47,629 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-06-09 12:17:47,630 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-06-09 12:17:47,631 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-06-09 12:17:47,631 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-06-09 12:17:47,631 INFO  [utils.coun

NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
NOT_DEFINED
TBD
TBD
NOT_DEFINED
NFPORS_20250407_point


2025-06-09 12:17:48,548 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-06-09 12:17:48,548 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-06-09 12:17:48,548 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-06-09 12:17:48,552 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-06-09 12:17:48,553 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-06-09 12:17:48,555 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-06-09 12:17:48,556 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category is 'Watershed Improvement'
2025-06-09 12:17:48,557 INFO  [utils.counts_to_mas   ]              counts step 7/8: set to 'NO' if Agency is 'Other' and Admin is 'CARB'


...

USFS_Region05_enriched_20250606


2025-06-09 12:19:55,997 INFO  [pyogrio._io           ]  Created 401,999 records
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-06-09 12:20:01,795 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-06-09 12:20:01,795 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-06-09 12:20:01,796 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-06-09 12:20:01,796 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-06-09 12:20:01,797 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-06-09 12:20:01,797 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-06-09 12:20:01,798 INFO  [utils.co

NOT_DEFINED
NOT_DEFINED
USFS_Region06_enriched_20250606


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-06-09 12:20:01,982 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-06-09 12:20:01,983 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-06-09 12:20:01,983 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (1950-2025)
2025-06-09 12:20:01,984 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-06-09 12:20:01,984 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-06-09 12:20:01,985 INFO  [utils.counts_to_mas   ]              counts step 5/8: set to 'NO' if status is 'Canceled', 'Planned', 'Outyear', or 'Proposed'
2025-06-09 12:20:01,986 INFO  [utils.counts_to_mas   ]              counts step 6/8: set to 'NO' if Activity Category i

USFS_Region04_enriched_20250606


In [12]:
enriched_polygons, enriched_lines, enriched_points = get_enriched_features(enriched_layers)

2025-06-09 14:22:11,703 INFO  [process.append_polygon]  --------------------------------------------------------------------------------
2025-06-09 14:22:11,703 INFO  [process.append_polygon]  Concatenate all polygon records
2025-06-09 14:22:11,703 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'BLM_enriched_20250425' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\BLM_2020_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-06-09 14:22:11,751 INFO  [process.append_polygon]  Load GeoDataFrame from the layer 'CNRA_enriched_20250421_polygon' in 'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb' 
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-06-09 14:22:14,597 INFO 

In [ ]:
# temp fix for CNRA MRCA/SMMC admin org assignment
# all MRCA reported data although populated to be MRCA by CNRA, comes from SMMC activity
enriched_polygons.loc[enriched_polygons.ADMINISTERING_ORG == 'MRCA', :] = 'SMMC'
enriched_polygons.loc[enriched_polygons.ADMINISTERING_ORG == 'MRCA', :] = 'SMMC'
enriched_polygons.loc[enriched_polygons.ADMINISTERING_ORG == 'MRCA', :] = 'SMMC'

In [14]:
import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [15]:
# grab timber non spatial path again
timber_nonspatial_path = None
for p in enriched_layers['point']:
    if 'Timber_Nonspatial' in p['gdb_path']:
        timber_nonspatial_path = p
        break

In [16]:
timber_nonspatial = gpd.read_file(timber_nonspatial_path['gdb_path'], 
                                    driver='OpenFileGDB', 
                                    layer=timber_nonspatial_path['layer_name'])

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [17]:
# temp fix for timber pipeline not assigning admin org
timber_nonspatial.ADMINISTERING_ORG = timber_nonspatial.AGENCY

In [18]:
%%time
for df, lyr_name in zip([enriched_polygons,enriched_lines,enriched_points], ["appended_poly","appended_line","appended_point"]):
    # init dask gdf for multithread clipping
    ddf = dask_geopandas.from_geopandas(df, npartitions=16)
    # clip to california bounds
    append_clipped = ddf.sjoin(california_boundary, how='inner', predicate='intersects').compute()
    # drop unwanted artifact columns from California boundary df
    append_clipped = append_clipped.drop(['index_right', 'Shape_Area', 'Shape_Length'], axis=1)
    
    # industry nonspatial is by design out of california bounds and got clipped, manually concat it back
    if lyr_name == 'appended_point':
        append_clipped = pd.concat([append_clipped, timber_nonspatial], ignore_index=True)
    save_gdf_to_gdb(append_clipped, append_path, lyr_name)

2025-06-09 14:27:52,197 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-06-09 14:27:52,198 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-06-09 14:27:52,230 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\appended.gdb


2025-06-09 14:28:18,529 INFO  [pyogrio._io           ]  Created 434,138 records
2025-06-09 14:28:25,007 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\appended.gdb appended_poly
2025-06-09 14:29:37,754 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-06-09 14:29:37,754 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-06-09 14:29:37,761 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\appended.gdb


2025-06-09 14:29:53,384 INFO  [pyogrio._io           ]  Created 103,837 records
2025-06-09 14:29:56,486 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\appended.gdb appended_line
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<timed exec>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future 

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\appended.gdb


2025-06-09 14:29:58,678 INFO  [pyogrio._io           ]  Created 14,415 records
2025-06-09 14:29:58,783 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\appended.gdb appended_point


CPU times: total: 45min 38s
Wall time: 7min 11s


In [1]:
append_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\appended.gdb"

In [5]:
gpd.list_layers(append_path)

,name,geometry_type
0,appended_poly,MultiPolygon
1,appended_line,MultiLineString
2,appended_point,Point


In [6]:
enriched_polygons = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_poly')
enriched_points = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_point')
enriched_lines = gpd.read_file(append_path, driver='OpenFileGDB', layer='appended_line')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [7]:
excel_out = pd.concat([enriched_points, enriched_lines, enriched_polygons])

In [8]:
excel_out.drop(['geometry'], axis=1).to_csv('its_all.csv')

In [20]:
from datetime import datetime

In [21]:
def get_activity_report(enriched_points, enriched_lines, enriched_polygons):
    append_all = pd.concat([enriched_lines, enriched_points, enriched_polygons])
    append_all = append_all[(append_all['COUNTS_TO_MAS'] == 'YES')]
    
    append_all.geometry = gpd.points_from_xy(append_all['LONGITUDE'],append_all['LATITUDE'])
    
    
    append_all = append_all[["AGENCY",
        "ADMINISTERING_ORG",
        "PRIMARY_OWNERSHIP_GROUP",
        "COUNTY",
        "REGION",
        "ACTIVITY_DESCRIPTION",
        "ACTIVITY_CAT",
        "BROAD_VEGETATION_TYPE",
        "ACTIVITY_STATUS",
        "ACTIVITY_QUANTITY",
        "ACTIVITY_UOM",
        "ACTIVITY_END",
        "Year_txt",
        "geometry"
    ]]
    
    # check if geometry is_valid
    # personally this is redundent, unless lat, lon is not valid, but that would throw an error in previous part
    append_all = append_all[append_all.is_valid]
    
    
    def get_entity_type(agency):
        if agency in ['CALEPA', 'CALSTA', 'CNRA', 'PARKS', 'California State Parks']:
            return 'State'
        if agency in ['DOD', 'DOI', 'USDA', 'DOE', 'NPS']:
            return 'Federal'
        if agency in ['Industrial Timber', 'Timber Companies', 'TIMBER']:
            return 'Timber Companies'
        else:
            return None
        
        
    append_all['ENTITY_TYPE'] = append_all['AGENCY'].apply(get_entity_type)
    
    return append_all

In [22]:
activity_report_gdf = get_activity_report(enriched_points, enriched_lines, enriched_polygons)

In [23]:
save_gdf_to_gdb(activity_report_gdf, 
                report_path, 
                f'activity_report{datetime.today().strftime('%Y%m%d')}')

2025-06-09 14:30:38,646 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-06-09 14:30:38,647 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-06-09 14:30:38,671 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\reports.gdb


2025-06-09 14:30:44,195 INFO  [pyogrio._io           ]  Created 454,512 records
2025-06-09 14:30:46,767 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\reports.gdb activity_report20250609


In [7]:
activity_report_gdf = gpd.read_file(report_path, driver='OpenFileGDB', layer='activity_report20250609')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [8]:
activity_report_gdf[activity_report_gdf.Year_txt == '2023'].groupby(['ADMINISTERING_ORG']).sum('ACTIVITY_QUANTITY')

,ACTIVITY_QUANTITY
ADMINISTERING_ORG,
BIA,8145.000000
BLM,24639.000000
CALFIRE,157584.621500
CALTRANS,135940.980700
CCC,1019.862762
CDFW,117383.086539
DOD,1883.230000
DOE,7186.925000
FWS,80618.000000


In [9]:
cnra_path = {'gdb_path': 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\ITSGDB_backup\\tmp\\CNRA_2020_2025.gdb',
  'layer_name': 'CNRA_enriched_20250421_polygon'}

In [10]:
cnra_gdf = gpd.read_file(cnra_path['gdb_path'],
                        driver='OpenFileGDB',
                        layer=cnra_path['layer_name'])

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts.  The processing may be really slow.  You can skip the processing by setting METHOD=SKIP.
  return ogr_read(


In [28]:
cnra_gdf[(cnra_gdf.ADMINISTERING_ORG == 'MRCA') & (cnra_gdf.Year_txt == '2023')].ORG_ADMIN_t

200     SMMC
202     SMMC
204     SMMC
206     SMMC
208     SMMC
        ... 
1058    SMMC
1059    SMMC
1060    SMMC
1090    SMMC
1093    SMMC
Name: ORG_ADMIN_t, Length: 256, dtype: object

In [14]:
pfirs_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\PFIRS_2020_2025.gdb"
PFIRS_20250402'

In [15]:
pfirs_gdf = gpd.read_file(pfirs_path, driver='OpenFileGDB', layer=layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [17]:
pfirs2024_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0\PFIRS_1950_2025.gdb"
layer_name = 'PFIRS_20250523'

In [18]:
pfirs2024_gdf = gpd.read_file(pfirs2024_path, driver='OpenFileGDB', layer=layer_name)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [48]:
gdf = pfirs2024_gdf

In [49]:
mask_parks = ~((gdf['ADMINISTERING_ORG'] == 'PARKS') & (gdf['TRMTID_USER'].apply(lambda x: x[:5] == 'PFIRS' if isinstance(x, str) else False)))

In [55]:
pfirs_gdf[pfirs_gdf['ADMINISTERING_ORG'] == 'OTHER'].AGENCY.unique()

array(['OTHER'], dtype=object)

In [56]:
pfirs_gdf[pfirs_gdf['AGENCY'] == 'OTHER'].ADMINISTERING_ORG.unique()

array(['OTHER'], dtype=object)

In [50]:
pfirs_gdf[(pfirs_gdf.ADMINISTERING_ORG == 'PARKS') 
          & (pfirs_gdf.Year_txt=='2023') 
          & (pfirs_gdf.COUNTS_TO_MAS=='YES')].ACTIVITY_QUANTITY.sum()

np.float64(2469.75)

In [57]:
pfirs_gdf[(pfirs_gdf.ADMINISTERING_ORG == 'PARKS')]

,PROJECTID_USER,AGENCY,ORG_ADMIN_p,PROJECT_CONTACT,PROJECT_EMAIL,ADMINISTERING_ORG,PROJECT_NAME,PROJECT_STATUS,PROJECT_START,PROJECT_END,...,Source,Year,Year_txt,Act_Code,Crosswalk,Federal_FY,State_FY,TRMT_GEOM,COUNTS_TO_MAS,geometry
54,PFIRS-116,None,PARKS,Jason Branz,jason.branz@arb.ca.gov,PARKS,Tamarisk Grove Debris project,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Hand Pile Burn,2023,2023,POINT,YES,POINT (338373.016 -535265.432)
55,PFIRS-117,None,PARKS,Jason Branz,jason.branz@arb.ca.gov,PARKS,Tamarisk Grove Debris project,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Hand Pile Burn,2023,2023,POINT,YES,POINT (338373.016 -535265.432)
56,PFIRS-118,None,PARKS,Jason Branz,jason.branz@arb.ca.gov,PARKS,Tamarisk Grove Debris project,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Hand Pile Burn,2023,2023,POINT,YES,POINT (338373.016 -535265.432)
57,PFIRS-119,None,PARKS,Jason Branz,jason.branz@arb.ca.gov,PARKS,Tamarisk Grove Debris project,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Hand Pile Burn,2023,2023,POINT,YES,POINT (338373.016 -535265.432)
58,PFIRS-120,None,PARKS,Jason Branz,jason.branz@arb.ca.gov,PARKS,Tamarisk Grove Debris project,None,NaT,NaT,...,PIFIRS,2023.0,2023,NaN,Hand Pile Burn,2023,2023,POINT,YES,POINT (338373.016 -535265.432)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11470,PFIRS-19479,None,PARKS,Jason Branz,jason.branz@arb.ca.gov,PARKS,Upper Prairie Prescribed Fire,None,NaT,NaT,...,PIFIRS,2019.0,2019,NaN,Broadcast Burn,2020,2020,POINT,YES,POINT (-286697.305 66748.733)
11471,PFIRS-19480,None,PARKS,Jason Branz,jason.branz@arb.ca.gov,PARKS,Upper Prairie Prescribed Fire,None,NaT,NaT,...,PIFIRS,2019.0,2019,NaN,Broadcast Burn,2020,2020,POINT,YES,POINT (-286697.305 66748.733)
11472,PFIRS-19481,None,PARKS,Jason Branz,jason.branz@arb.ca.gov,PARKS,Upper Prairie Prescribed Fire,None,NaT,NaT,...,PIFIRS,2020.0,2020,NaN,Broadcast Burn,2021,2021,POINT,YES,POINT (-286697.305 66748.733)
11473,PFIRS-19482,None,PARKS,Jason Branz,jason.branz@arb.ca.gov,PARKS,Upper Prairie Prescribed Fire,None,NaT,NaT,...,PIFIRS,2020.0,2020,NaN,Broadcast Burn,2021,2021,POINT,YES,POINT (-286697.305 66748.733)


In [42]:
mask_parks

0        True
1        True
2        True
3        True
4        True
         ... 
12170    True
12171    True
12172    True
12173    True
12174    True
Length: 12175, dtype: bool

In [61]:
activity_report_gdf[activity_report_gdf.ADMINISTERING_ORG == 'TIMBER']

,AGENCY,ADMINISTERING_ORG,PRIMARY_OWNERSHIP_GROUP,COUNTY,REGION,ACTIVITY_DESCRIPTION,ACTIVITY_CAT,BROAD_VEGETATION_TYPE,ACTIVITY_STATUS,ACTIVITY_QUANTITY,ACTIVITY_UOM,ACTIVITY_END,Year_txt,ENTITY_TYPE,geometry
105246,<NA>,TIMBER,PRIVATE_INDUSTRY,PLA,SIERRA_NEVADA,PILE_BURN,PRESCRIBED_FIRE,FOREST,COMPLETE,0.000000,AC,2017-11-14 00:00:00+00:00,2017,None,POINT (-3791.341 143348.615)
105247,<NA>,TIMBER,PRIVATE_INDUSTRY,PLA,SIERRA_NEVADA,PILE_BURN,PRESCRIBED_FIRE,FOREST,COMPLETE,5.000000,AC,2018-01-17 00:00:00+00:00,2018,None,POINT (-3791.341 143348.615)
105248,<NA>,TIMBER,FEDERAL,SIE,SIERRA_NEVADA,PILE_BURN,PRESCRIBED_FIRE,SHRB_CHAP,COMPLETE,0.000000,AC,2017-11-20 00:00:00+00:00,2017,None,POINT (-4468.485 165574.121)
105249,<NA>,TIMBER,FEDERAL,SIE,SIERRA_NEVADA,PILE_BURN,PRESCRIBED_FIRE,SHRB_CHAP,COMPLETE,0.000000,AC,2018-01-17 00:00:00+00:00,2018,None,POINT (-4468.485 165574.121)
105250,<NA>,TIMBER,PRIVATE_INDUSTRY,PLA,SIERRA_NEVADA,PILE_BURN,PRESCRIBED_FIRE,FOREST,COMPLETE,7.000000,AC,2017-11-14 00:00:00+00:00,2017,None,POINT (-4912.244 142126.766)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107799,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,TREE_PLNTING,TREE_PLNTING,FOREST,COMPLETE,40000.000000,AC,2022-12-31 00:00:00+00:00,2022,Timber Companies,POINT (-293152.972 -48482.165)
107800,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,TREE_PLNTING,TREE_PLNTING,FOREST,COMPLETE,34337.536889,AC,2023-12-31 00:00:00+00:00,2023,Timber Companies,POINT (-293152.972 -48482.165)
107801,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,VARIABLE_RETEN_HARVEST,TIMB_HARV,FOREST,COMPLETE,136.500000,AC,2021-12-31 00:00:00+00:00,2021,Timber Companies,POINT (-284207.655 -50310.285)
107802,TIMBER,TIMBER,PRIVATE_INDUSTRY,None,None,VARIABLE_RETEN_HARVEST,TIMB_HARV,FOREST,COMPLETE,273.000000,AC,2022-12-31 00:00:00+00:00,2022,Timber Companies,POINT (-284207.655 -50310.285)


In [37]:
lookup_table_path = r'D:\WORK\wildfire\Interagency-Tracking-System\2023\PFIRS_2023\pfirs_agency_2022_2023_AT.xlsx'
pfirs_lookup_df = pd.read_excel(lookup_table_path)


# construct dictionary from lookup table for agency name remaping and anonymous timber company parsing
pfirs_lookup_dict = pd.Series(pfirs_lookup_df.agency_calc_field_2.values,index=pfirs_lookup_df.Agency).to_dict()
anonymous_org_list = pfirs_lookup_df[pfirs_lookup_df["Org Type"] == "Industrial Timber"].Agency.tolist()